In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/Measurement_summary', './input/Measurement_item_info', './input/Measurement_station_info']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

이 커널은 Just Kaggle 스터디의 일환으로

[bappe](https://www.kaggle.com/bappekim)님의 [Air Pollution in Seoul](https://www.kaggle.com/bappekim/air-pollution-in-seoul) 데이터셋을 살펴보는 커널입니다.

In [1]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove plotting
# import seaborn as sns
# import matplotlib.pyplot as plt

# ALEX: remove path printing
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# 데이터 불러오기

In [2]:
df_summary = pd.read_csv('./input/Measurement_summary.scaled.csv')
df_item = pd.read_csv('./input/Measurement_item_info.scaled.csv')
df_station = pd.read_csv('./input/Measurement_station_info.scaled.csv')

In [3]:
df_summary.head()

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0


### Measurement date를 date와 time으로 나누고, 일 평균 값으로 합침

In [4]:
date_time = df_summary['Measurement date'].str.split(" ", n=1, expand=True)
date_time.head()

,0,1
0,2017-01-01,00:00
1,2017-01-01,01:00
2,2017-01-01,02:00
3,2017-01-01,03:00
4,2017-01-01,04:00


In [5]:
df_summary['date'] = date_time[0]
df_summary['time'] = date_time[1]
df_summary = df_summary.drop(['Measurement date'], axis=1)
df_summary.head()

,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,date,time
0,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,2017-01-01,00:00
1,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0,2017-01-01,01:00
2,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0,2017-01-01,02:00
3,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0,2017-01-01,03:00
4,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0,2017-01-01,04:00


### 일단 서울 전체에 대해서 분석해보기 위해서 date로 groupby하고 분석

In [6]:
df_seoul = df_summary.groupby(['date'], as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})
df_seoul.head()

,date,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01,0.003627,0.044765,0.002478,0.981833,77.201667,56.773333
1,2017-01-02,0.002707,0.035960,0.013127,0.891333,109.243333,77.838333
2,2017-01-03,0.000602,0.037017,0.008223,0.753833,78.546667,51.533333
3,2017-01-04,0.004122,0.048813,0.006918,0.878500,54.966667,34.533333
4,2017-01-05,0.003122,0.033892,0.009725,0.656333,36.246667,22.168333


In [7]:
# ALEX: remove plotting
# df_seoul.plot(x='date')

In [8]:
corr = df_seoul.corr()
# ALEX: remove plotting
# f, ax = plt.subplots(figsize=(11, 9))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(corr, cmap=cmap, vmax=1, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

/tmp/ipykernel_392587/3686643608.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_seoul.corr()


### 미세먼지(PM10) 기준으로 좋음, 보통, 나쁨, 매우나쁨으로 구분

대한민국의 미세먼지 환경기준(일평균)
- 좋음 0~30
- 보통 ~80
- 나쁨 ~150
- 매우나쁨151~

In [9]:
df_seoul['PM10_class'] = -1
df_seoul.head()

,date,SO2,NO2,O3,CO,PM10,PM2.5,PM10_class
0,2017-01-01,0.003627,0.044765,0.002478,0.981833,77.201667,56.773333,-1
1,2017-01-02,0.002707,0.035960,0.013127,0.891333,109.243333,77.838333,-1
2,2017-01-03,0.000602,0.037017,0.008223,0.753833,78.546667,51.533333,-1
3,2017-01-04,0.004122,0.048813,0.006918,0.878500,54.966667,34.533333,-1
4,2017-01-05,0.003122,0.033892,0.009725,0.656333,36.246667,22.168333,-1


In [10]:
for (idx, row) in df_seoul.iterrows():
    pm10 = row[5]
    _class = -1
    if pm10 < 0:
        continue
    elif pm10 < 30:
        _class = 0
    elif pm10 < 80:
        _class = 1
    elif pm10 < 150:
        _class = 2
    else:
        _class = 3
    df_seoul.loc[idx, 'PM10_class'] = _class
df_seoul.head()

,date,SO2,NO2,O3,CO,PM10,PM2.5,PM10_class
0,2017-01-01,0.003627,0.044765,0.002478,0.981833,77.201667,56.773333,1
1,2017-01-02,0.002707,0.035960,0.013127,0.891333,109.243333,77.838333,2
2,2017-01-03,0.000602,0.037017,0.008223,0.753833,78.546667,51.533333,1
3,2017-01-04,0.004122,0.048813,0.006918,0.878500,54.966667,34.533333,1
4,2017-01-05,0.003122,0.033892,0.009725,0.656333,36.246667,22.168333,1


In [11]:
# ALEX: remove plotting
# df_seoul['PM10_class'].value_counts().plot(kind="bar")
df_seoul['PM10_class'].value_counts()

1    664
0    344
2     69
3      8
Name: PM10_class, dtype: int64

일부로 기준을 그렇게 정했겠지만,

예상과 다르게 보통 > 좋음 > 나쁨 > 매우나쁨 순서로 나타남.

### Examine Strongest Correlation

In [12]:
# ALEX: remove plotting
# sns.jointplot(x=df_seoul["CO"], y=df_seoul["NO2"], kind='kde', xlim=(0,1),ylim=(0,0.13), color='g')
# plt.show()
_ = df_seoul["CO"]
_ = df_seoul["NO2"]